## This is a notebook to recombine NER entities into one

In [1]:
import pandas as pd
import re
import numpy as np

In [12]:
df = pd.read_csv("landmine_tweets_ner_results_eng.csv")

In [13]:
df['tagged_text'] = df['tagged_text'].fillna(" ")

In [14]:
df['tagged_text'] = df["tagged_text"].str.replace("##", "")

In [15]:
def combine_ner_tags(df):
    combined_tokens = []
    cur_tokens = [df.loc[0, "tagged_text"]]
    for i in range(1, len(df)):
        if df.loc[i, "ner_tag"].startswith("B-"):
            combined_tokens.append(cur_tokens)
            cur_tokens = [df.loc[i, "tagged_text"]]
        elif df.loc[i, "ner_tag"].startswith("I-"):
             cur_tokens.append(df.loc[i, "tagged_text"])
    b_tags = df[df["ner_tag"].str.startswith("B-")]
    combined_tokens.append([b_tags.tail(1)["tagged_text"].values[0]])
    # update the combined text join based on which language. no space for non_eng
    combined_text = [' '.join(i) for i in combined_tokens]
    b_tags["tagged_text"] = combined_text
    return b_tags

In [16]:
df_combined = combine_ner_tags(df)

<ipython-input-15-3cce659609bf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [19]:
df_combined.tail()

,document_id,sentence_number,sentence_text,tagged_text,ner_tag,tagged_text_loci,ner_model,probability
8112,1635232000050331648,1,kashmir indian forces claim to have busted an ...,kashmir,B-group,0,tner/bert-large-tweetner7-2020,0.999973
8113,1635232000050331648,1,kashmir indian forces claim to have busted an ...,indian forces,B-group,1,tner/bert-large-tweetner7-2020,0.971510
8115,1635232000050331648,1,kashmir indian forces claim to have busted an ...,islamabad anantnag,B-location,14,tner/bert-large-tweetner7-2020,0.999986
8117,1635232000050331648,1,kashmir indian forces claim to have busted an ...,9x19mm ammunition pakistani india shakir bashir,B-product,27,tner/bert-large-tweetner7-2020,0.999962
8123,1635336636186906624,1,the syrian desert is a known jihadist hotspot ...,syrian,B-location,1,tner/bert-large-tweetner7-2020,0.999950


In [20]:
df_combined.shape

(3534, 8)

In [21]:
df_combined["tagged_text"]

0                                            scooby doo
2                                     ashura procession
4                                        balkh district
6                                        balkh province
8                                      sayed ul shuhada
                             ...                       
8112                                            kashmir
8113                                      indian forces
8115                                 islamabad anantnag
8117    9x19mm ammunition pakistani india shakir bashir
8123                                             syrian
Name: tagged_text, Length: 3534, dtype: object

In [22]:
df_combined.to_csv("landmine_tweets_ner_results_eng_clean.csv", index = False)